# The following are analysis performed for this Bachelor Thesis

In [16]:
%load_ext autoreload
%autoreload 2

from tabulate import tabulate
from texttable import Texttable
import latextable
import header as head
import json
import os

annotations_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations"
community_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection"
bridges_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


There are 74 rumour source tweet with 0 non-rumour source tweet within the total of 74 rumour tweet.

In [ ]:
# p = head.getPath()
# print(p)
head.count()

In [ ]:
# merging following relationship into a JSON file
head.followRelate()

with open('/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json') as f:
    data = json.load(f)
    followRelate_Ids = list(data.keys())

In [ ]:
# generate dictionary for tweetId and userId conversion
# results are saved in Id-conversions folder
head.gen_idConversion()

There are followers who reacted to the source tweets. 
* reacts from followers are saved in `react_follows`
* reacts from non-followers are saved in `react_not_follows`

In [ ]:
# See whether there are any reactors that follows the person they responded to 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json") as f:
    followingRelate = json.load(f)

head.is_follower_react(followingRelate)

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/with_bridges.json") as file:
    communities = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    who_follows_whom = json.load(file)

bridges = communities["bridges"]
rumours = communities["rumours"]
non_rumours = communities["non_rumours"]

bridges_follow = {}

for bridge in bridges:
    follows = who_follows_whom[bridge]
    bridges_follow[bridge] = {
        "rumours":[],
        "non_rumours":[],
        "bridges":[]
    }

    for f in follows:
        if f in rumours: (bridges_follow[bridge])["rumours"].append(f)
        elif f in non_rumours: (bridges_follow[bridge])["non_rumours"].append(f)
        elif f in bridges: (bridges_follow[bridge])["bridges"].append(f)

head.writeToJSON(head.getRoot(),"bridges_follows",bridges_follow)


Exporting tabular to file

In [28]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/centrality.json") as file:
    bridge_centrality = json.load(file)

table = Texttable()
table.set_cols_align(["c", "c", "c"])
table.set_cols_valign(["t", "m", "b"])
table.add_rows([["id","#rumour followers","#non-rumour followers"]])

data = []
for id in bridge_centrality:
    bridge = bridge_centrality[id]
    info = [id,bridge["rum_count"],bridge["nonrum_count"]]
    data.append(info)

table.add_rows(data)
with open('follower_counts.txt', 'w') as f:
    f.write(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))



In [27]:
table_1 = Texttable()
table_1.set_cols_align(["l", "r", "c"])
table_1.set_cols_valign(["t", "m", "b"])
table_1.add_rows([["Name", "Age", "Nickname"],
                    ["Mr\nXavier\nHuon", 32, "Xav'"],
                    ["Mr\nBaptiste\nClement", 1, "Baby"],
                    ["Mme\nLouise\nBourgeau", 28, "Lou\n \nLoue"]])
print('-- Example 1: Basic --')
print('Texttable Output:')
print(table_1.draw())
print('\nLatextable Output:')
print(latextable.draw_latex(table_1, caption="An example table.", label="table:example_table"))

-- Example 1: Basic --
Texttable Output:
+----------+-----+----------+
|   Name   | Age | Nickname |
+==========+=====+==========+
| Mr       |     |          |
| Xavier   |  32 |          |
| Huon     |     |   Xav'   |
+----------+-----+----------+
| Mr       |     |          |
| Baptiste |   1 |          |
| Clement  |     |   Baby   |
+----------+-----+----------+
| Mme      |     |   Lou    |
| Louise   |  28 |          |
| Bourgeau |     |   Loue   |
+----------+-----+----------+

Latextable Output:
\begin{table}
	\begin{center}
		\begin{tabular}{|l|r|c|}
			\hline
			Name & Age & Nickname \\
			\hline
			MrXavierHuon & 32 & Xav' \\
			\hline
			MrBaptisteClement & 1 & Baby \\
			\hline
			MmeLouiseBourgeau & 28 & Lou Loue \\
			\hline
		\end{tabular}
	\end{center}
	\caption{An example table.}
	\label{table:example_table}
\end{table}
